# Calculates b-values and time lags

This notebook calculates b-values as well as time lags for the data. B-values are calculated per a Flinn-Engdahl region. B-value is calculated from the magnitudes of earthquakes, it is the relationship between earthquake magnitude and frequency of occurrence. B-values are typically close to 1.0 because for every whole number increase in magnitude, there is a earthquake occurances at that magitude decrease by log 10. This implies that if a region has a high b-value there has been a disproportional number of low magitude earthquakes and the region is due for a larger magitude event. 

In [1]:
source('initialize_data.R', echo = FALSE)

In [2]:
install.packages('spdep')

load_libraries()

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

The following object is masked from ‘package:purrr’:

    transpose

Loading required package: sp
### Welcome to rworldmap ###
For a short introduction type : 	 vignette('rworldmap')
Checking rgeos availability: TRUE

Attaching package: ‘maps’

The fo

[1] "libraries loaded successfully"


In [3]:
#Explicit location of main project data
path <- "~/jupyter/cs2019_Group11/GroupProducts/data" 

#create df from main project data
df <- loadFiles(path) %>%
    reqCols %>%
    parseDt('time')

#create spatial point df
dfsp <- tospdf(df)

Files in "~/jupyter/cs2019_Group11/GroupProducts/data" loaded.
Subset Completed.
Parsed column: time.
spatial df created


In [4]:
#takes about 10 minutes to run
#adds Flinn-Engdahl regions to spatial point df and converts coordinate system
tpts <-regions_data(dfsp)

OGR data source with driver: LIBKML 
Source: "/dsa/home/jaw56m/jupyter/cs2019_Group11/GroupProducts/fe.kmz", layer: "fe"
with 754 features
It has 12 fields


Warning message in readOGR(tectonicdata):
“Z-dimension discarded”

[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"
[1] "none"


In [5]:
base_world <- base_world_regions()
#get base map

OGR data source with driver: LIBKML 
Source: "/dsa/home/jaw56m/jupyter/cs2019_Group11/GroupProducts/fe.kmz", layer: "fe"
with 754 features
It has 12 fields


Warning message in readOGR(regiondata):
“Z-dimension discarded”Regions defined for each Polygons
Warning message:
“Ignoring unknown parameters: fill”

In [6]:
tpts_df <- as.data.frame(tpts)
tpts_df$latitude = df$latitude
tpts_df$longitude = df$longitude
tpts_df$coords.x1 = NULL
tpts_df$coords.x2 = NULL
head(tpts_df)

depth,mag,time,id,updated,newname,latitude,longitude
2.000,2.6,1999-07-01 23:29:27,usp0009awp,2014-11-07 01:08:01,374,45.4500,3.790
4.613,3.1,1999-07-01 22:10:51,uw10474128,2016-07-23 18:59:57,6,46.7105,-122.778
106.800,4.2,1999-07-01 21:42:41,usp0009awk,2014-11-07 01:08:01,1,51.9950,177.972
26.100,3.9,1999-07-01 20:50:11,usp0009awh,2014-11-07 01:08:01,53,-31.7260,-72.053
136.300,3.1,1999-07-01 19:52:19,usp0009awf,2014-11-07 01:08:01,24,59.8900,-153.560
30.200,3.8,1999-07-01 19:52:12,usp0009awe,2014-11-07 01:08:01,741,36.0500,31.166


A ID column must be added to the Flinn-Engdahl regions in order for the loop that will be ran later to know how many iterations to run.

In [7]:
tectonicdata = "~/jupyter/cs2019_Group11/GroupProducts/fe.kmz"
    tectonicFeatures <- readOGR(tectonicdata)

    #also transform data into same regions as the earthquake data for later comparisons
    transTectonicFeatures <- spTransform(tectonicFeatures,  CRS("+init=epsg:4087"))


tectonicFeatures@data$ID <- seq.int(nrow(transTectonicFeatures))#this adds in an id which is the same 
#as the newname in the tpts to match regions with earthquakes



OGR data source with driver: LIBKML 
Source: "/dsa/home/jaw56m/jupyter/cs2019_Group11/GroupProducts/fe.kmz", layer: "fe"
with 754 features
It has 12 fields


Warning message in readOGR(tectonicdata):
“Z-dimension discarded”

Empty values are created to store the output of the loop later.

In [8]:
bvalue = " "
error = " "
count = " "
time_diff = " "
year = " "
listofdfs <- list()

Now the b-values are ready to be calculated.

In [9]:
#jessica

options(warn=-1) # there are warnings because some regions don't have earthquakes 

fe = as.data.frame(tectonicFeatures) #convert to use in the loop 

for (n in 1999:2018){
    time = subset(tpts_df, format(as.Date(tpts_df$time),"%Y")==n)
    print('substed')
    
for (i in 1:nrow(fe)) {  # i need this to run for every region 

    region <- subset(tpts_df, newname == i) # i pull out all earthquake per a region 

    if (length(region) >= 1){ # keeps from getting errors because some errors have no earthquakes 
        values <- b.guten(region$mag) # i calc the bvalues using the function 
        values_df = as.data.frame(values)# must be converted to a df to pull out the numbers 
        bvalue [i] <- values_df[1,] # pull out the bvalues 
        error [i] <- values_df[2,] # pull out the errors 
        count[i] = i+1 # save the count, it represents the region 
        year[i] = n
} 
    year[n] = n
    data  = as.data.frame(cbind(bvalue, error, count, year))
}
    #data[n]  = as.data.frame(c(bvalue, error, count))
     #year[n] = n
 listofdfs[[n]] <- assign(paste("bvalue_year",n[1999:2018],sep="_"),data) 
 
}



[1] "substed"
[1] "substed"
[1] "substed"
[1] "substed"
[1] "substed"
[1] "substed"
[1] "substed"
[1] "substed"
[1] "substed"
[1] "substed"
[1] "substed"
[1] "substed"
[1] "substed"
[1] "substed"
[1] "substed"
[1] "substed"
[1] "substed"
[1] "substed"
[1] "substed"
[1] "substed"


In [10]:
#values = 1999:2018
#wanted = listofdfs[values]

df_bvalues = bind_rows(listofdfs)

head(df_bvalues)

bvalue,error,count,year
0.508351114613314,0.0101603726882616,2,1999
0.584190090008462,0.00770723358770015,3,1999
0.525083031942309,0.0639985271831257,4,1999
1.00474746934992,0.118692367680826,5,1999
0.632058592679377,0.0639401903928455,6,1999
1.05761413285105,0.0317995238777704,7,1999


Now that b-values are calculated, data must be converted to be numeric and put into a dataframe.

In [11]:
df_bvalues$bvalue[is.nan(df_bvalues$bvalue)] <- 0

df_bvalues$bvalue = as.numeric(df_bvalues$bvalue)

df_bvalues$error[is.nan(df_bvalues$error)] <- 0

df_bvalues$error = as.numeric(df_bvalues$error)
#test2[is.nan(test2)] <- 0

df_bvalues$year = as.numeric(df_bvalues$year)

newname = as.data.frame(df_bvalues$count) # converting the counts to put into a df 
df_bvalues$newname <- (as.integer(df_bvalues$count))

#b_value = cbind(test1, test2, newname) # i put all together into a df that way i know the region later  
#b_value$variable = NULL
df_bvalues$count = NULL

head(df_bvalues)

bvalue,error,year,newname
0.5083511,0.010160373,1999,2
0.5841901,0.007707234,1999,3
0.5250830,0.063998527,1999,4
1.0047475,0.118692368,1999,5
0.6320586,0.063940190,1999,6
1.0576141,0.031799524,1999,7


The b-values are now ready to be added onto the earthquake dataframe

In [12]:
 list_values = list()
tpts_df$year = format(as.Date(tpts_df$time),"%Y")

for (i in 1999:2018){
    year_tpts = subset (tpts_df, year == i)
    year_bvalues = subset (df_bvalues, year == i)

    df_part <- merge(year_tpts, year_bvalues, by.x='newname', by.y='newname')
     list_values[[i]] <- assign(paste("merged",i[1999:2018],sep="_"),df_part) 

}

merged_all = bind_rows(list_values)
merged_all$year.x = NULL
merged_all$year.y = NULL

head(merged_all)

newname,depth,mag,time,id,updated,latitude,longitude,bvalue,error
10,33.0,5.1,1999-02-14 17:17:27,usp00092tz,2014-11-07 01:07:01,21.581,-106.681,0.6204207,0.66570086
10,33.0,4.5,1999-03-16 06:14:07,usp00094kr,2014-11-07 01:07:14,22.002,-107.375,0.6204207,0.66570086
10,10.0,4.5,1999-01-13 07:38:50,usp0009114,2014-11-07 01:06:48,22.690,-107.988,0.6204207,0.66570086
10,10.0,5.3,1999-03-12 07:07:40,usp00094by,2016-11-09 22:04:11,22.219,-107.368,0.6204207,0.66570086
10,33.0,5.2,1999-01-10 20:36:06,usp00090x3,2016-11-09 21:38:36,22.939,-108.020,0.6204207,0.66570086
100,129.1,4.3,1999-04-15 14:05:21,usp00096bk,2014-11-07 01:07:28,-19.295,-69.268,0.3347563,0.04778564


Now the time lag between earthquakes is calculated. Lags are determined for earthquakes within the same Flinn-Engdahl regions.  

In [13]:
library(dplyr)
data <- 
    merged_all %>%
    group_by(newname) %>%
    mutate(time_lag = dplyr::lag(time, n = 1, default = NA))

head(data)


newname,depth,mag,time,id,updated,latitude,longitude,bvalue,error,time_lag
10,33.0,5.1,1999-02-14 17:17:27,usp00092tz,2014-11-07 01:07:01,21.581,-106.681,0.6204207,0.66570086,NA
10,33.0,4.5,1999-03-16 06:14:07,usp00094kr,2014-11-07 01:07:14,22.002,-107.375,0.6204207,0.66570086,1999-02-14 17:17:27
10,10.0,4.5,1999-01-13 07:38:50,usp0009114,2014-11-07 01:06:48,22.690,-107.988,0.6204207,0.66570086,1999-03-16 06:14:07
10,10.0,5.3,1999-03-12 07:07:40,usp00094by,2016-11-09 22:04:11,22.219,-107.368,0.6204207,0.66570086,1999-01-13 07:38:50
10,33.0,5.2,1999-01-10 20:36:06,usp00090x3,2016-11-09 21:38:36,22.939,-108.020,0.6204207,0.66570086,1999-03-12 07:07:40
100,129.1,4.3,1999-04-15 14:05:21,usp00096bk,2014-11-07 01:07:28,-19.295,-69.268,0.3347563,0.04778564,NA


In [14]:
big = data %>%
        group_by(newname) %>%
        summarize(mean_mag = mean(mag), std_mag = sd(mag),  bvalue = mean(bvalue), error = mean(error), 
                  max_mag = max(mag))
#Group by year in order to get stats per year, want mag and depth as they are the only interesting things to look at

big = arrange(big, desc(big$max_mag))
#desc order to better understand the larger magnitude events

names(big)[1]<-"Flinn-Engdahl region"
names(big)[2]<-"Mean Magnitude per region"
names(big)[4]<-"B-value per region"
names(big)[5]<-"B-value error per region"
names(big)[6]<-"Maximum Magitude earthquake"



#rename columns for easy of reader
#have some basic univairate stats to look at
head(big, 10)

Flinn-Engdahl region,Mean Magnitude per region,std_mag,B-value per region,B-value error per region,Maximum Magitude earthquake
197,4.545749,0.4685629,0.3161679,0.015341091,9.1
570,4.535332,0.4325227,0.5825635,0.054201369,9.1
54,3.777263,0.7496326,0.2824985,0.007453174,8.8
593,4.545121,0.4292538,0.3373858,0.074676802,8.6
191,4.638098,0.4695939,0.4387210,0.031523097,8.4
65,4.533912,0.5145688,0.4687103,0.059476348,8.4
171,4.525546,0.4437419,0.2822652,0.014144272,8.3
553,4.372358,0.6804321,0.8474039,0.454624977,8.3
669,4.429820,0.5047722,0.3047233,0.004853625,8.3
60,4.288951,0.3990974,0.3546653,0.021544392,8.2



For the regions with the highest magitude events all have relatively low b-values. This indicates that these regions are due for some lower magitude earthquakes.

In [15]:
write.csv(data,'data.csv')
df = read.csv("data.csv",header=TRUE,sep=",")
head(df)


X,newname,depth,mag,time,id,updated,latitude,longitude,bvalue,error,time_lag
1,10,33.0,5.1,1999-02-14 17:17:27,usp00092tz,2014-11-07 01:07:01,21.581,-106.681,0.6204207,0.66570086,NA
2,10,33.0,4.5,1999-03-16 06:14:07,usp00094kr,2014-11-07 01:07:14,22.002,-107.375,0.6204207,0.66570086,1999-02-14 17:17:27
3,10,10.0,4.5,1999-01-13 07:38:50,usp0009114,2014-11-07 01:06:48,22.690,-107.988,0.6204207,0.66570086,1999-03-16 06:14:07
4,10,10.0,5.3,1999-03-12 07:07:40,usp00094by,2016-11-09 22:04:11,22.219,-107.368,0.6204207,0.66570086,1999-01-13 07:38:50
5,10,33.0,5.2,1999-01-10 20:36:06,usp00090x3,2016-11-09 21:38:36,22.939,-108.020,0.6204207,0.66570086,1999-03-12 07:07:40
6,100,129.1,4.3,1999-04-15 14:05:21,usp00096bk,2014-11-07 01:07:28,-19.295,-69.268,0.3347563,0.04778564,NA


In [16]:
summary(df)

       X             newname        depth             mag       
 Min.   :     1   Min.   :  2   Min.   : -3.60   Min.   :2.500  
 1st Qu.:126551   1st Qu.: 98   1st Qu.: 10.00   1st Qu.:3.000  
 Median :253100   Median :189   Median : 21.82   Median :4.000  
 Mean   :253100   Mean   :290   Mean   : 54.64   Mean   :3.802  
 3rd Qu.:379650   3rd Qu.:514   3rd Qu.: 49.80   3rd Qu.:4.500  
 Max.   :506200   Max.   :754   Max.   :735.80   Max.   :9.100  
                                                                
     time                id              updated             latitude      
 Length:506200      Length:506200      Length:506200      Min.   :-84.422  
 Class :character   Class :character   Class :character   1st Qu.: -5.463  
 Mode  :character   Mode  :character   Mode  :character   Median : 19.647  
                                                          Mean   : 18.417  
                                                          3rd Qu.: 40.361  
                        